# Value at Risk

---

El **Valor en Riesgo** (Value at Risk o VaR) es una de las medidas de riesgo más utilizadas en finanzas. Esta medida busca medir el riesgo de mercado en un portafolio.

Matemáticamente, el VaR hace referencia a un cuantil de la distrubición de los rendimientos de un portafolio o, de manera más general, la distribución de nuestra variable de interés.

Recordemos que un cuantil de nivel $\alpha$ de la distribución $F_{X}$, se define como el número $q_{\alpha}$ tal que

$$ F_{X}(q_{\alpha}) = \mathbb{P}[X \leq q_{\alpha}] = \alpha$$

El VaR se define como una medida de riesgo (estadística) que estima la máxima pérdida que puede presentar un portafolio dado un nivel de confianza $\alpha$.

Si la variable aleatoria $X$ representa los rendimientos de un portafolio (esto implica que $X \in (-\infty, \infty)$), el VaR con un nivel de confianza $\alpha$, es el número $VaR_{X}(\alpha)$ tal que:

$$\mathbb{P}[X \leq VaR_{X}(\alpha)] = 1- \alpha$$

Por ejemplo si $\alpha = 0.95$, entonces el $VaR_{X}(0.95)$ se puede interpretar como la pérdida que se tendría en a lo más el $5\%$ de los casos.

Podemos ver así que el $VaR_{X}(\alpha)$ representa el cuantil $1 - \alpha$ de la distribución de $X$.

In [2]:
import numpy as np
from scipy.stats import norm
np.random.seed(54321)

#Datos a utilizar
datos = np.random.normal(loc = 0, scale = 0.2, size = 50000)

## VaR Paramétrico
---

Suponga que $X \sim N(\mu, \sigma)$ es una variable aleatoria que representa los rendimientos diarios de un portafolio.

Podemos calcular el $VaR_{X}(\alpha)$ de la siguiente manera:

$$
\begin{array}{ccl}
\mathbb{P}[X \leq VaR_{X}(\alpha)] & = & 1 - \alpha \\
\mathbb{P}\left[Z \leq \dfrac{VaR_{X}(\alpha) - \mu}{\sigma}\right] & = & 1 - \alpha \\
VaR_{X}(\alpha) & = & \mu + \sigma \Phi^{-1}(1 - \alpha) \\
 & = & \mu - \sigma \Phi^{-1}(\alpha) \\
\end{array}
$$

en donde $Z$ es una variable aleatoria normal, $\Phi^{-1}$ es la función inversa de la función de distribución de una variable aleatoria normal.

**Ejercicio**
Utilizando el desarrollo anterior, programe una función que estime el var paramétrico para un conjunto de observaciones.

**Sugerencia** 
Lea la documentación de 
```python
help(norm.ppf)
help(np.std)
help(np.mean)
```

In [3]:
#Escriba aquí su código

## VaR Histórico (no paramétrico)

A diferencia del VaR paramétrico, el VaR histórico no asume ninguna distribución para los rendimientos de un portafolio.

Su cálculo se realiza utilizando los cuantiles empíricos de los datos dados.

**Ejercicio**  Lea la documentación de
```python
help(np.quantile)
```
y programe una función que calcule el VaR histórico. Compare sus resultados con aquellos obtenidos con el VaR paramétrico.

**Sugerencia** Recuerde la definición de un cuantil $q_\alpha$ así como la interpretación del VaR.

In [ ]:
#Escriba su código aquí

## Conditional Value at Risk (CVaR)

A diferencia del VaR, el CVaR toma en cuenta la forma de la distribución de los rendimientos.

Esta medida puede interpretarse como la pérdida promedio esperada, cuando los rendimientos del portafolio son menores o iguales al VaR. En otras palabras, si $X$ es una variable aleatoria que representa los rendimientos de un portafolio y $\alpha$ el nivel de confianza, el CVaR está dado por:

$$
\begin{array}
CVaR_{X}(\alpha) & = & \mathbb{E}[X | X \leq VaR_{X}(\alpha)] \\
& \approx & \dfrac{1}{|A|} \sum_{x_{i} \in A} x_{i}
\end{array}
$$

en donde $A = \{ x : x \leq VaR_{X}(\alpha) \}$.

**Ejercicio** Programe una función que calcule el CVaR de un conjunto de datos.



In [19]:
# Escriba aquí su código